뉴스 감성분석에 필요한 데이터 중, 긍정 레이블로 분류된 기사들을 LDA(잠재 디리클레 할당) 분석기법을 통해 토픽별로 모델링한 결과입니다.

# 필요 패키지 로드

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# 데이터 전처리

In [2]:
#데이터 불러오기
data1 = pd.read_csv('data_1121.csv', encoding = 'utf-8')

In [3]:
#데이터 정보 확인
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10977 entries, 0 to 10976
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  10977 non-null  int64 
 1   index       10977 non-null  int64 
 2   icls_cnts   10977 non-null  object
 3   label       10977 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 343.2+ KB


In [4]:
#필요 레이블만 추출, value 별 확인
data1 = data1[['label','icls_cnts']]
data1['label'].value_counts()

 0    7681
 1    2087
-1    1209
Name: label, dtype: int64

In [5]:
#중복된 행 제거
data1.drop_duplicates(subset = ['icls_cnts'], inplace = True)
data1['label'].value_counts()

 0    6820
 1    2061
-1    1185
Name: label, dtype: int64

In [6]:
#긍정 레이블만 추출
df1 = data1[data1['label']==1]

In [7]:
#결측값 여부 확인
print('결측값 여부:',df1.isnull().values.any())

결측값 여부: False


In [8]:
import re 
#텍스트 정제 함수 : 한글 이외의 문자 제거
def text_cleaning(text):
    #한글 정규표현식으로 한글만 추출
    hangul = re.compile('[^ㄱ-ㅣ가-힣]+')
    result = hangul.sub(' ',text)
    return result

In [9]:
#함수 적용
df1['ko_text']=df1['icls_cnts'].apply(lambda x: text_cleaning(x))
df1.head

/tmp/ipykernel_260104/4292822634.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['ko_text']=df1['icls_cnts'].apply(lambda x: text_cleaning(x))


<bound method NDFrame.head of        label                                          icls_cnts  \
482        1  "전동 바운서"에 장시간 태우는 등 학대해 숨지게 한 혐의로 20대 친모가 <b>경...   
484        1  생후 9개월 영아 탈수·영양실조 상태로 심정지…30대 친모 조사 심정지 상태로 병원...   
501        1  사상구 아동학대 예....<b>경찰</b>서, 북부교육지원청, 서부아동보호전문기관,...   
511        1  부산 중구 ..‘생물 테러 대비·대응 대규모 .. 이날 훈련에는 부산시, 중부<b>...   
537        1  광주 도심 집단 난투극 외국인 일부 입건 광주 도심 한복판에서 심야 집단 난투극을 ...   
...      ...                                                ...   
10971      1  ..<b>경찰</b>에 붙잡혔다.....<b>경찰</b>청 광역<b>수사</b>대는...   
10972      1  경남 합천<b>경찰</b>서, 전화금융사기 예방 금융 직원에 감사장 수여 경남 합천...   
10973      1  ‘베스트 여청<b>수사</b>팀’ 선정 김해서부<b>경찰</b>서(서장 심태환)은 ...   
10974      1  김영환 영주<b>경찰</b>서 부청문감사인권관, 경북<b>경찰</b>청 베스트 팀장...   
10976      1  <b>경찰</b>, 승용차 절도 10대 추격전 끝 검거 ..[KBS 청주] ..훔친...   

                                                 ko_text  
482     전동 바운서 에 장시간 태우는 등 학대해 숨지게 한 혐의로 대 친모가 경찰 에 붙...  
484    생후 개월 영아 탈수 영양실조 상태로 심정지 대 친모 조사 심정지 상태로

In [10]:
#pos tagging 함수
def get_pos(x):
    tagger = Okt()
    pos = tagger.pos(x)
    pos = ['{}/{}'.format(word,tag) for word, tag in pos]
    return pos

In [11]:
#상위 100개의 단어를 보존
#TF-IDF 행렬을 만들기 
vectorizer = TfidfVectorizer(max_features = 100)
X = vectorizer.fit_transform(df1['ko_text'])

#행렬 확인
print('TF-IDF 행렬의 크기:', X.shape)

TF-IDF 행렬의 크기: (2061, 100)


In [12]:
#LDA 토픽 모델링
lda_model = LatentDirichletAllocation(n_components =10, learning_method = 'online', random_state = 777, max_iter =1)
lda_top = lda_model.fit_transform(X)

print(lda_model.components_)
print(lda_model.components_.shape)

[[ 2.03579839  0.17630237  0.19181942 22.92172006  2.33529644  0.23869175
   0.50867016 75.99909977  4.86665825  0.17396097  0.94966595  0.59947643
   0.64243289  0.17477855  6.82760299  0.4501382  14.34861238  9.97980942
  20.30417367  8.75197439 17.00807094  0.38567493 10.76686277  4.26473576
   0.16702859 43.80648378  5.98437697  1.59304856 17.22917922 27.11932465
   5.12754898  7.99077582  0.42204199 13.193133    0.26049651  0.56564643
   0.72513601 11.80605855  0.24942235  0.20616673 13.02614076  4.70600711
  37.54032771  2.64522012  1.70415047  3.09124198  3.02277604  6.04796462
   1.59185699  0.17543383  0.32072125  0.26949895 11.99511377  1.95317644
   0.28205945  4.86894392  2.53473811  7.23800911  1.98193119  0.4348862
  30.22925099  1.23739832  5.91998046  3.93717698  3.10048988  0.18359101
   0.18807803  4.786473    7.60813486  1.63218236  2.46379548  0.25311801
   0.20032841  0.67845977  0.18109392 12.89186909  8.69248356  7.48986273
   1.60149654  4.09606993  0.20865814  

In [13]:
#토픽 추출 및 확인
terms = vectorizer.get_feature_names()

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" %(idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:n -1:-1]])
        
get_topics(lda_model.components_,terms)

Topic 1: [('경찰', 76.0), ('명을', 43.81), ('수사', 37.54), ('위반', 30.23), ('범죄', 27.12), ('혐의로', 26.13), ('했다', 24.51), ('검거', 22.92), ('등에', 20.3), ('밝혔다', 17.23), ('등의', 17.01), ('대한', 14.35), ('붙잡혔다', 13.19), ('서울', 13.03), ('지난', 12.89), ('통해', 12.45), ('여성', 12.0), ('서는', 11.81), ('따르면', 10.77), ('대해', 9.98), ('등을', 8.75), ('지난달', 8.69), ('부산', 7.99), ('있다', 7.61), ('지난해', 7.49), ('오후', 7.24), ('기자', 6.83), ('씨를', 6.05), ('명이', 5.98), ('위해', 5.92), ('보이스피싱', 5.13), ('예정이다', 4.87), ('관계자는', 4.87), ('있는', 4.79), ('서장', 4.71), ('최근', 4.37), ('말했다', 4.26), ('직원', 4.1), ('이날', 3.94), ('이번', 3.1), ('씨가', 3.09), ('씨는', 3.02), ('신고', 2.65), ('청은', 2.62), ('오전', 2.53), ('자치', 2.46), ('것으로', 2.34), ('가운데', 2.04), ('함께', 2.02), ('올해', 1.98), ('예방', 1.95), ('신고를', 1.7), ('있도록', 1.63), ('지역', 1.6), ('받고', 1.59), ('아동학대', 1.59), ('청장', 1.43), ('코로나', 1.38), ('위한', 1.24), ('관련', 0.95), ('서가', 0.73), ('조사', 0.68), ('피해를', 0.65), ('교통', 0.64), ('광주', 0.6), ('사고', 0.57), ('경북', 0.51), ('대상으로', 0.45), ('

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
